# Notebook to extract hidden-states and attention heads activations from gpt2 model predictions

In [1]:
import os
import glob
import torch
import gc
import numpy as np
import pandas as pd
from tqdm import tqdm
from model import GPT2Extractor
from sklearn.preprocessing import StandardScaler
from tokenizer import tokenize
from utils import set_seed
from numpy import linalg as la
import matplotlib.pyplot as plt

In [2]:
def check_folder(path):
    """Create adequate folders if necessary."""
    try:
        if not os.path.isdir(path):
            check_folder(os.path.dirname(path))
            os.mkdir(path)
    except:
        pass

In [3]:
def transform(activations, path, name, run_index, n_layers_hidden=13, n_layers_attention=12, hidden_size=768):
    assert activations.values.shape[1] == (n_layers_hidden + n_layers_attention) * hidden_size
    indexes = [[index*hidden_size, (index+1)*hidden_size] for index in range(n_layers_hidden + n_layers_attention)]
    for order in [2]: # np.inf
        matrices = []
        for i, index in enumerate(indexes):
            matrix = activations.values[:, index[0]:index[1]]
            #with_std = True if order=='std' else False
            #scaler = StandardScaler(with_mean=True, with_std=with_std)
            #scaler.fit(matrix)
            #matrix = scaler.transform(matrix)
            if order is not None and order != 'std':
                matrix = matrix / np.mean(la.norm(matrix, ord=order, axis=1))
            matrices.append(matrix)
        matrices = np.hstack(matrices)
        new_data = pd.DataFrame(matrices, columns=activations.columns)
        new_path = path + '_norm-' + str(order).replace('np.', '')
        check_folder(new_path)
        new_data.to_csv(os.path.join(new_path, name + '_run{}.csv'.format(run_index + 1)), index=False)


Defining variables:

In [4]:
template = '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/text/english/text_english_run*.txt' # path to text input
language = 'english'

Creating iterator for each run:

In [5]:
template = '/Users/alexpsq/Code/Parietal/data/text_english_run*.txt' # path to text input


In [6]:
paths = sorted(glob.glob(template))

In [7]:
iterator_list = [tokenize(path, language, train=False) for path in paths]

100%|██████████| 207/207 [00:00<00:00, 967916.31it/s]


In [8]:
#import utils
#import seaborn as sns
#import matplotlib.pyplot as plt
#from transformers import GPT2Tokenizer
#
#tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
#
#lengths = []
#
#for index in range(9):
#    batches, indexes = utils.batchify_per_sentence_with_context(
#                iterator_list[index], 
#                1, 
#                5, 
#                'gpt2', 
#                max_length=512)
#    #lengths.append(np.array(sorted([len(item.split()) for item in batches])))
#    lengths.append(np.array(sorted([len(tokenizer.tokenize(item)) for item in batches])))
#
#    sns.boxplot(lengths[-1])
#    plt.show()
#    print()
#
#print(np.mean(np.array([np.mean(item) for item in lengths])))
#print(np.median(np.array([np.median(item) for item in lengths])))
#print(np.mean(np.array([np.median(item) for item in lengths])))

In [9]:
#from transformers import GPT2Model
#GPT2Model.from_pretrained('gpt2').config

In [10]:
#import utils
#config = {'stop_attention_at_sent': None,
#          'number_of_sentence': 1,
#          'stop_attention_before_sent': 0
#         }
#
#batches, indexes = utils.batchify_with_detailed_indexes(
#            iterator_list[0], 
#            config['number_of_sentence'], 
#            20, 
#            'gpt2', 
#            1024,
#            config['stop_attention_at_sent'],
#            config['stop_attention_before_sent'],
#    True
#        )
#
#
#
#indexes_tmp = [(indexes[i][-config['number_of_sentence']][0], indexes[i][-1][1]) for i in range(len(indexes))]
#indexes_tmp[0] = (indexes[0][0][0], indexes[0][-1][1])
#print(indexes_tmp)
#for i in indexes_tmp:
#    print(i[1]-i[0])
#
#
#
#from transformers import GPT2Tokenizer, GPT2Model
#tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
#model = GPT2Model.from_pretrained('gpt2', output_hidden_states=True)
#
#m = 0
#hidden_states_activations = []
#
#for index, batch in enumerate(batches):
#    batch = batch.strip() # Remove trailing character
#    #batch = '<|endoftext|> ' + batch + ' <|endoftext|>'
#    
#    tokenized_text = tokenizer.tokenize(batch, add_prefix_space=False)
#    inputs_ids = torch.tensor([tokenizer.convert_tokens_to_ids(tokenized_text)])
#    attention_mask = torch.tensor([[1 for x in tokenized_text]])
#
#    if (config['stop_attention_at_sent'] is not None) and (index > 0):
#        attention_mask[:, :1 + indexes[index][-config['stop_attention_at_sent']-config['number_of_sentence']][0]] = 0
#        if self.config['stop_attention_before_sent'] < 0:
#            attention_mask[:, 1 + indexes[index][-config['stop_attention_at_sent']-config['number_of_sentence']][0]: 1 + indexes[index][-config['stop_attention_at_sent']-config['number_of_sentence']][0]-config['stop_attention_before_sent']] = 0
#        elif self.config['stop_attention_before_sent'] > 0:
#            attention_mask[:, 1 + indexes[index][-config['stop_attention_at_sent']-config['number_of_sentence']][0]-config['stop_attention_before_sent']: 1 + indexes[index][-config['stop_attention_at_sent']-config['number_of_sentence']][0]] = 1
#    mapping = utils.match_tokenized_to_untokenized(tokenized_text, batch)
#    
#    with torch.no_grad():
#        encoded_layers = model(inputs_ids, attention_mask=attention_mask) 
#    hidden_states_activations_ = np.vstack(encoded_layers[2]) # retrieve all the hidden states (dimension = layer_count * len(tokenized_text) * feature_count)
#    a = utils.extract_activations_from_token_activations(hidden_states_activations_, mapping, indexes_tmp[index])
#    #m += len(tokenized_text[indexes_tmp[index][0]:indexes_tmp[index][1]])
#    m += len(a)
#    #print(len(a), a[0].shape)
#    #print(len(tokenized_text), indexes_tmp[index][0], indexes_tmp[index][1])
#    #print(tokenized_text[indexes_tmp[index][0]:indexes_tmp[index][1]])
#    #print(np.array(tokenized_text)[attention_mask.detach().numpy()[0].astype(bool)])
#    #print(mapping)
#    key_start = None
#    key_stop = None
#    for key_, value in mapping.items(): 
#        if (value[0])== (indexes_tmp[index][0]): #because we added <|endoftext|> token at the beginning
#            key_start = key_
#
#    #print(key_start, len(mapping.keys()))
#    #print(tokenized_text)
#    for word_index in range(key_start, len(mapping.keys())): # len(mapping.keys()) - 1
#        print([index for index in mapping[word_index]])
#    #print()
#    #a=input()
#    #if a!='a':
#    #    break
#print(m)

## Activation extraction

In [14]:
pretrained_gpt2_models = ['gpt2'] 
names = [
    'gpt2_pre-5_1_token-2'
        ]
config_paths = [None] * 56
saving_path_folders = [
    '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/stimuli-representations/{}/gpt2_pre-5_1_token-2'.format(language)
    
]
prediction_types = ['sentence'] * 56
number_of_sentence_list = [1] * 56
number_of_sentence_before_list = [2]
attention_length_before_list = [2] * 56
stop_attention_at_sent_list = [None] * 56
stop_attention_before_sent_list = [0] * 56


In [15]:
output_attentions = False
output_hidden_states = True

In [16]:
for index, gpt2_model in enumerate(pretrained_gpt2_models):
    extractor = GPT2Extractor(gpt2_model, 
                              language, 
                              names[index], 
                              prediction_types[index],
                              output_hidden_states=output_hidden_states, 
                              output_attentions=output_attentions,
                              attention_length_before=attention_length_before_list[index],
                              config_path=config_paths[index],
                              max_length=512, 
                              number_of_sentence=number_of_sentence_list[index], 
                              number_of_sentence_before=number_of_sentence_before_list[index],
                              stop_attention_at_sent=stop_attention_at_sent_list[index],
                              stop_attention_before_sent=stop_attention_before_sent_list[index]
                             )
    print(extractor.name, ' - Extracting activations ...')
    for run_index, iterator in tqdm(enumerate(iterator_list)):
        gc.collect()
        print("############# Run {} #############".format(run_index + 1))
        activations  = extractor.extract_activations(iterator, language)
        hidden_states_activations = activations[0]
        attention_heads_activations = activations[1]
        #activations = pd.concat([hidden_states_activations, attention_heads_activations], axis=1)
        print(hidden_states_activations.shape)
        #transform(
        #    hidden_states_activations, 
        #    saving_path_folders[index], 
        #    'activations', 
        #    run_index=run_index,
        #    n_layers_hidden=13,
        #    n_layers_attention=0, 
        #    hidden_size=768)
        
        #transform(cls_activations, saving_path_folders[index], 'cls')
        #transform(sep_activations, saving_path_folders[index], 'sep')
        
        #check_folder(saving_path_folders[index])
        #hidden_states_activations.to_csv(os.path.join(saving_path_folders[index], 'activations_run{}.csv'.format(run_index + 1)), index=False)
        #activations.to_csv(os.path.join(saving_path_folders[index], 'activations_run{}.csv'.format(run_index + 1)), index=False)
        #cls_activations.to_csv(os.path.join(saving_path_folders[index], 'cls_run{}.csv'.format(run_index + 1)), index=False)
        #sep_activations.to_csv(os.path.join(saving_path_folders[index], 'sep_run{}.csv'.format(run_index + 1)), index=False)
        del activations
        del hidden_states_activations
        #del attention_heads_activations

0it [00:00, ?it/s]

gpt2_pre-5_1_token-2  - Extracting activations ...
############# Run 0 #############
 Once ,  when  I  was  six  years  old ,  I  saw  a  magnificent  picture  in  a  book  about  the  primeval  forest  called  ‘  Real  -  life  Stories .  ’  It  showed  a  boa  constrictor  swallowing  a  wild  animal .
 Here  is  a  copy  of  the  drawing .
 It  said  in  the  book  :  “  Boa  constrictors  swallow  their  prey  whole ,  without  chewing .
 Then  they  are  not  able  to  move ,  and  they  sleep  for  the  six  months  it  takes  for  digestion .  ”
 So  I  thought  a  lot  about  the  adventures  of  the  jungle  and ,  in  turn ,  I  managed ,  with  a  coloured  pencil ,  to  make  my  first  drawing .
 My  Drawing  Number  one .
 It  looked  like  this  :  I  showed  my  masterpiece  to  the  grownups  and  I  asked  them  if  my  drawing  frightened  them .
 They  answered  me  :  “  Why  would  anyone  be  frightened  by  a  hat ?  ”
 My  drawing  was  not  of  a  hat .
 It  s

0it [00:03, ?it/s]


KeyboardInterrupt: 

In [ ]:
for index, bert_model in enumerate(pretrained_bert_models):
    extractor = BertExtractor(bert_model, 
                              language, 
                              names[index], 
                              prediction_types[index], 
                              output_hidden_states=True, 
                              output_attentions=True, 
                              config_path=config_paths[index], 
                              max_length=512, 
                              number_of_sentence=number_of_sentence_list[index], 
                              number_of_sentence_before=number_of_sentence_before_list[index], 
                              number_of_sentence_after=number_of_sentence_after_list[index])
    print(extractor.name, ' - Extracting activations ...')
    for run_index, iterator in tqdm(enumerate(iterator_list)):
        print("############# Run {} #############".format(run_index))
        activations  = extractor.extract_activations(iterator, language)
        hidden_states_activations = activations[0]
        attention_heads_activations = activations[1]
        (cls_hidden_states_activations, cls_attention_activations) = activations[2]
        (sep_hidden_states_activations, sep_attention_activations) = activations[3]
        activations = pd.concat([hidden_states_activations, attention_heads_activations], axis=1)
        cls_activations = pd.concat([cls_hidden_states_activations, cls_attention_activations], axis=1)
        sep_activations = pd.concat([sep_hidden_states_activations, sep_attention_activations], axis=1)
        
        transform(activations, saving_path_folders[index], 'activations', run_index=run_index)
        #transform(cls_activations, saving_path_folders[index], 'cls')
        #transform(sep_activations, saving_path_folders[index], 'sep')
        
        #activations.to_csv(os.path.join(saving_path_folders[index], 'activations_run{}.csv'.format(run_index + 1)), index=False)
        #cls_activations.to_csv(os.path.join(saving_path_folders[index], 'cls_run{}.csv'.format(run_index + 1)), index=False)
        #sep_activations.to_csv(os.path.join(saving_path_folders[index], 'sep_run{}.csv'.format(run_index + 1)), index=False)
        del activations
        del cls_activations
        del sep_activations
        del hidden_states_activations
        del attention_heads_activations
        del cls_hidden_states_activations
        del cls_attention_activations
        del sep_hidden_states_activations
        del sep_attention_activations

In [ ]:
extractor.model

In [ ]:
for p in list(extractor.model.named_parameters()):
    print(p[0])

### Generate control activations

In [ ]:
bert_model = 'gpt2'
language = 'english'
name = 'gpt2_control_'
prediction_type = 'sentence'
saving_path_folder = '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/stimuli-representations/{}'.format(language)
seeds = [24, 213, 1111, 61, 183]

In [ ]:
def randomize_layer(model, layer_nb):
    """Randomize layer weights and put bias to zero.
    The input "layer_nb" goes from 1 to 12 to be coherent with the rest of the analysis.
    It is then transfomed in the function.
    """
    layer_nb = layer_nb - 1
    model.h[layer_nb].ln_1.weight = torch.nn.parameter.Parameter(torch.rand_like(model.h[layer_nb].ln_1.weight))
    model.h[layer_nb].ln_1.bias = torch.nn.parameter.Parameter(torch.zeros_like(model.h[layer_nb].ln_1.bias))
    model.h[layer_nb].attn.c_attn.weight = torch.nn.parameter.Parameter(torch.rand_like(model.h[layer_nb].attn.c_attn.weight))
    model.h[layer_nb].attn.c_attn.bias = torch.nn.parameter.Parameter(torch.zeros_like(model.h[layer_nb].attn.c_attn.bias))
    model.h[layer_nb].attn.c_proj.weight = torch.nn.parameter.Parameter(torch.rand_like(model.h[layer_nb].attn.c_proj.weight))
    model.h[layer_nb].attn.c_proj.bias = torch.nn.parameter.Parameter(torch.zeros_like(model.h[layer_nb].attn.c_proj.bias))
    model.h[layer_nb].ln_2.weight = torch.nn.parameter.Parameter(torch.rand_like(model.h[layer_nb].ln_2.weight))
    model.h[layer_nb].ln_2.bias = torch.nn.parameter.Parameter(torch.zeros_like(model.h[layer_nb].ln_2.bias))
    model.h[layer_nb].mlp.c_fc.weight = torch.nn.parameter.Parameter(torch.rand_like(model.h[layer_nb].mlp.c_fc.weight))
    model.h[layer_nb].mlp.c_fc.bias = torch.nn.parameter.Parameter(torch.zeros_like(model.h[layer_nb].mlp.c_fc.bias))
    model.h[layer_nb].mlp.c_proj.weight = torch.nn.parameter.Parameter(torch.rand_like(model.h[layer_nb].mlp.c_proj.weight))
    model.h[layer_nb].mlp.c_proj.bias = torch.nn.parameter.Parameter(torch.zeros_like(model.h[layer_nb].mlp.c_proj.bias))
    return model

In [ ]:
def randomize_ln_1(model, layer_nb):
    """Randomize attention query weights of a given layer and put bias to zero.
    """
    layer_nb = layer_nb - 1
    model.h[layer_nb].ln_1.weight = torch.nn.parameter.Parameter(torch.rand_like(model.h[layer_nb].ln_1.weight))
    model.h[layer_nb].ln_1.bias = torch.nn.parameter.Parameter(torch.zeros_like(model.h[layer_nb].ln_1.bias))
    return model

def randomize_attention_c_attn(model, layer_nb):
    """Randomize attention key weights of a given layer and put bias to zero.
    """
    layer_nb = layer_nb - 1
    model.h[layer_nb].attn.c_attn.weight = torch.nn.parameter.Parameter(torch.rand_like(model.h[layer_nb].attn.c_attn.weight))
    model.h[layer_nb].attn.c_attn.bias = torch.nn.parameter.Parameter(torch.zeros_like(model.h[layer_nb].attn.c_attn.bias))
    return model

def randomize_attention_c_proj(model, layer_nb):
    """Randomize attention value weights of a given layer and put bias to zero.
    """
    layer_nb = layer_nb - 1
    model.h[layer_nb].attn.c_proj.weight = torch.nn.parameter.Parameter(torch.rand_like(model.h[layer_nb].attn.c_proj.weight))
    model.h[layer_nb].attn.c_proj.bias = torch.nn.parameter.Parameter(torch.zeros_like(model.h[layer_nb].attn.c_proj.bias))
    return model

def randomize_ln_2(model, layer_nb):
    """Randomize attention dense network weights of a given layer and put bias to zero.
    """
    layer_nb = layer_nb - 1
    model.h[layer_nb].ln_2.weight = torch.nn.parameter.Parameter(torch.rand_like(model.h[layer_nb].ln_2.weight))
    model.h[layer_nb].ln_2.bias = torch.nn.parameter.Parameter(torch.zeros_like(model.h[layer_nb].ln_2.bias))
    return model


def randomize_mlp_c_fc(model, layer_nb):
    """Randomize intermediate dense network weights of a given layer and put bias to zero.
    """
    layer_nb = layer_nb - 1
    model.h[layer_nb].mlp.c_fc.weight = torch.nn.parameter.Parameter(torch.rand_like(model.h[layer_nb].mlp.c_fc.weight))
    model.h[layer_nb].mlp.c_fc.bias = torch.nn.parameter.Parameter(torch.zeros_like(model.h[layer_nb].mlp.c_fc.bias))
    return model

def randomize_mlp_c_proj(model, layer_nb):
    """Randomize output dense network weights of a given layer and put bias to zero.
    """
    layer_nb = layer_nb - 1
    model.h[layer_nb].mlp.c_proj.weight = torch.nn.parameter.Parameter(torch.rand_like(model.h[layer_nb].mlp.c_proj.weight))
    model.h[layer_nb].mlp.c_proj.bias = torch.nn.parameter.Parameter(torch.zeros_like(model.h[layer_nb].mlp.c_proj.bias))
    return model


In [ ]:
def randomize_embeddings(model):
    """Randomize embeddings weights and put bias to zero.
    """
    model.wte.weight = torch.nn.parameter.Parameter(torch.rand_like(model.wte.weight))
    model.wpe.weight = torch.nn.parameter.Parameter(torch.rand_like(model.wpe.weight))
    return model

In [ ]:
for seed in seeds:
    set_seed(seed)
    for layer in range(13):
        extractor = GPT2Extractor(gpt2_model, language, name, prediction_type, output_hidden_states=True, output_attentions=True, config_path=None)
        if layer==0:
            extractor.model = randomize_embeddings(extractor.model)
        else:
            extractor.model = randomize_layer(extractor.model, layer)
        print(extractor.name + str(seed), ' - Extracting activations for layer {}...'.format(layer))
        for run_index, iterator in tqdm(enumerate(iterator_list)):
            print("############# Run {} #############".format(run_index))
            activations  = extractor.extract_activations(iterator, language)
            hidden_states_activations = activations[0]
            attention_heads_activations = activations[1]
            activations = pd.concat([hidden_states_activations, attention_heads_activations], axis=1)

            # activations
            heads = np.arange(1, 13)
            columns_to_retrieve = ['hidden_state-layer-{}-{}'.format(layer, i) for i in range(1, 769)]
            if layer > 0:
                columns_to_retrieve += ['attention-layer-{}-head-{}-{}'.format(layer, head, i) for head in heads for i in range(1, 65)]
            activations = activations[columns_to_retrieve]

            save_path = os.path.join(saving_path_folder, name + str(seed) + '_layer-{}'.format(layer))
            check_folder(save_path)
            print('\tSaving in {}.'.format(save_path))
            activations.to_csv(os.path.join(save_path, 'activations_run{}.csv'.format(run_index + 1)), index=False)


# Test activation extraction 

In [8]:
import utils 
import random
from transformers import GPT2Tokenizer

config = {
    'number_of_sentence': 1, 
    'number_of_sentence_before': 3, 
    'attention_length_before': 3, 
}



tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
extractor_full = GPT2Extractor('gpt2', 
                              'english', 
                              'test', 
                              'sentence', 
                              output_hidden_states=True, 
                              output_attentions=False, 
                              attention_length_before=config['attention_length_before'],
                              config_path=None, 
                              number_of_sentence=config['number_of_sentence'], 
                              number_of_sentence_before=config['number_of_sentence_before'], 
                             )
extractor_masked = GPT2Extractor('gpt2', 
                              'english', 
                              'test', 
                              'control-context', 
                              output_hidden_states=True, 
                              output_attentions=False, 
                              attention_length_before=config['attention_length_before'],
                              config_path=None, 
                              number_of_sentence=config['number_of_sentence'], 
                              number_of_sentence_before=config['number_of_sentence_before'], 
                             )
extractor_shuffle = GPT2Extractor('gpt2', 
                              'english', 
                              'test', 
                              'shuffle', 
                              output_hidden_states=True, 
                              output_attentions=False, 
                              attention_length_before=config['attention_length_before'],
                              config_path=None, 
                              number_of_sentence=config['number_of_sentence'], 
                              number_of_sentence_before=config['number_of_sentence_before'], 
                             )

In [17]:
# Full sentences
batches_full, indexes_full = utils.batchify_with_detailed_indexes(
            iterator_list[0], 
            config['number_of_sentence'], 
            config['number_of_sentence_before'], 
            'gpt2',
            add_prefix_space=True
        )

# Tokens are masked
batches_masked, indexes_masked = utils.batchify_with_detailed_indexes(
            iterator_list[0], 
            config['number_of_sentence'], 
            config['number_of_sentence_before'], 
            'gpt2',
            add_prefix_space=True
            )

# Shuffling
batches_shuffle, indexes_shuffle = utils.batchify_sentences(
            iterator_list[0], 
            config['number_of_sentence'], 
            config['number_of_sentence_before'], 
            pretrained_model='gpt2', 
            past_context_size=config['attention_length_before'],
            transformation='shuffle',
            vocabulary=None,
            dictionary=None,
            seed=1111,
            add_prefix_space=True
            )

In [18]:
# Preprocessing full
indexes_full_tmp = [(indexes_full[i][-config['number_of_sentence']][0], indexes_full[i][-1][1]) for i in range(len(indexes_full))]
indexes_full_tmp[0] = (indexes_full[0][0][0], indexes_full[0][-1][1])

for i in range(len(indexes_full_tmp)):
    indexes_full_tmp[i] = (indexes_full_tmp[i][0] + 1, indexes_full_tmp[i][1] + 1)
    
# Preprocessing masked
indexes_masked_tmp = [(indexes_masked[i][-config['number_of_sentence']][0], indexes_masked[i][-1][1]) for i in range(len(indexes_masked))]
indexes_masked_tmp[0] = (indexes_masked[0][0][0], indexes_masked[0][-1][1])

for i in range(len(indexes_masked_tmp)):
    indexes_masked_tmp[i] = (indexes_masked_tmp[i][0] + 1, indexes_masked_tmp[i][1] + 1)

# Preprocessing shuffled


In [43]:
# activation generation full
output = []
for index, batch in enumerate(batches_full):
    batch = batch.strip() # Remove trailing character
    batch = '<|endoftext|> ' + batch + ' <|endoftext|>'

    tokenized_text = tokenizer.tokenize(batch, add_prefix_space=False)
    mapping = utils.match_tokenized_to_untokenized(tokenized_text, batch)
    inputs_ids = torch.tensor([tokenizer.convert_tokens_to_ids(tokenized_text)])

    attention_mask = torch.tensor([[1 for x in tokenized_text]])

    with torch.no_grad():
        encoded_layers = extractor_full.model(inputs_ids, attention_mask=attention_mask) # last_hidden_state, pooler_output, hidden_states, attentions

        hidden_states_activations_ = np.vstack(encoded_layers[2]) # retrieve all the hidden states (dimension = layer_count * len(tokenized_text) * feature_count)
        
        #print(len(encoded_layers[2]))
        #print('output shape:', hidden_states_activations_.shape)
        
        new_activations = []
        key = None
        
        #print('Mapping:')
        #for key in mapping.keys():
        #    print(batch.split()[key], ''.join([tokenized_text[i] for i in mapping[key]]))
        #print('A priori Token of interest:', tokenized_text[indexes_full_tmp[index][0]:indexes_full_tmp[index][1]])
            
        for key_, value in mapping.items(): 
            if indexes_full_tmp[index][0] in value:
                key = key_
                
        #print(key)
        #print('dimension match:', len(tokenized_text)==hidden_states_activations_.shape[1])
        tmp = ' '.join([tokenizer.decode(tokenizer.convert_tokens_to_ids([tokenized_text[word] for word in mapping[index]])) for index in range(key, len(mapping.keys()) - 1)])
        tmp = tmp.replace('  ', ' ').strip()
        #print('Extracting sentence:')
        print(tmp)
        output.append(tmp)

        for word_index in range(key, len(mapping.keys()) - 1):
            word_activation = []
            word_activation.append([hidden_states_activations_[:,index, :] for index in mapping[word_index]])
            word_activation = np.vstack(word_activation)
            new_activations.append(np.mean(word_activation, axis=0).reshape(1,-1))

        
        #print(np.vstack(new_activations).shape)
        #if input()!='':
        #    break


assert ' '.join(output) == ' '.join(iterator_list[0])


Once , when I was six years old , I saw a magnificent picture in a book about the primeval forest called ‘ Real - life Stories . ’ It showed a boa constrictor swallowing a wild animal . Here is a copy of the drawing .
It said in the book : “ Boa constrictors swallow their prey whole , without chewing .
Then they are not able to move , and they sleep for the six months it takes for digestion . ”
So I thought a lot about the adventures of the jungle and , in turn , I managed , with a coloured pencil , to make my first drawing .
My Drawing Number one .
It looked like this : I showed my masterpiece to the grownups and I asked them if my drawing frightened them .
They answered me : “ Why would anyone be frightened by a hat ? ”
My drawing was not of a hat .
It showed a boa constrictor digesting an elephant .
I then drew the inside of the boa constrictor , so that the grownups could understand .
They always need to have things explained .
My Drawing Number two looked like this : The grownups 

In [44]:
# activation generation masked
output = []
for index_batch, batch in enumerate(batches_masked):
    batch = batch.strip() # Remove trailing character
    batch = '<|endoftext|> ' + batch + ' <|endoftext|>'

    tokenized_text = tokenizer.tokenize(batch, add_prefix_space=False)
    mapping = utils.match_tokenized_to_untokenized(tokenized_text, batch)

    beg = indexes_masked_tmp[index_batch][0] 
    end = indexes_masked_tmp[index_batch][1] 

    inputs_ids = torch.tensor([tokenizer.convert_tokens_to_ids(tokenized_text)])
    inputs_ids = torch.cat(inputs_ids.size(1) * [inputs_ids])
    inputs_ids = inputs_ids[beg:end, :]

    attention_mask =  torch.diag_embed(torch.tensor([0 for x in tokenized_text]))
    for i in range(min(len(tokenized_text), config['attention_length_before'])):
        attention_mask = torch.add(attention_mask, torch.diag_embed(torch.tensor([1 for x in range(len(tokenized_text) - i)]), offset=-i))
    attention_mask = attention_mask[beg:end, :]

    with torch.no_grad():
        encoded_layers = extractor_masked.model(inputs_ids, attention_mask=attention_mask) # last_hidden_state, pooler_output, hidden_states, attentions

        hidden_states_activations_ = np.vstack([torch.cat([encoded_layers[2][layer][i,len(tokenized_text) - encoded_layers[2][layer].size(0) + i - 1,:].unsqueeze(0) for i in range(encoded_layers[2][layer].size(0))], dim=0).unsqueeze(0).detach().numpy() for layer in range(len(encoded_layers[2]))])
        hidden_states_activations_ = np.concatenate([np.zeros((hidden_states_activations_.shape[0], indexes_masked_tmp[index_batch][0] , hidden_states_activations_.shape[-1])), hidden_states_activations_, np.zeros((hidden_states_activations_.shape[0], len(tokenized_text) - indexes_masked_tmp[index_batch][1], hidden_states_activations_.shape[-1]))], axis=1)
        # retrieve all the hidden states (dimension = layer_count * len(tokenized_text) * feature_count)
            
        #print(len(encoded_layers[2]))
        #print('output shape:', hidden_states_activations_.shape)
        
        new_activations = []
        key = None
        
        #print('Mapping:')
        #for key in mapping.keys():
        #    print(batch.split()[key], ''.join([tokenized_text[i] for i in mapping[key]]))
        #print('A priori Token of interest:', tokenized_text[indexes_masked_tmp[index_batch][0]:indexes_masked_tmp[index_batch][1]])
            
        for key_, value in mapping.items(): 
            if indexes_masked_tmp[index_batch][0] in value:
                key = key_
                
        #print(key)
        #print('dimension match:', len(tokenized_text)==hidden_states_activations_.shape[1])
        tmp = ' '.join([tokenizer.decode(tokenizer.convert_tokens_to_ids([tokenized_text[word] for word in mapping[index]])) for index in range(key, len(mapping.keys()) - 1)])
        tmp = tmp.replace('  ', ' ').strip()
        #print('Extracting sentence:')
        print(tmp)
        output.append(tmp)

        for word_index in range(key, len(mapping.keys()) - 1):
            word_activation = []
            word_activation.append([hidden_states_activations_[:,index, :] for index in mapping[word_index]])
            word_activation = np.vstack(word_activation)
            new_activations.append(np.mean(word_activation, axis=0).reshape(1,-1))

        
        #print(np.vstack(new_activations).shape)
        #if input()!='':
        #    break


assert ' '.join(output) == ' '.join(iterator_list[0])


Once , when I was six years old , I saw a magnificent picture in a book about the primeval forest called ‘ Real - life Stories . ’ It showed a boa constrictor swallowing a wild animal . Here is a copy of the drawing .
It said in the book : “ Boa constrictors swallow their prey whole , without chewing .
Then they are not able to move , and they sleep for the six months it takes for digestion . ”
So I thought a lot about the adventures of the jungle and , in turn , I managed , with a coloured pencil , to make my first drawing .
My Drawing Number one .
It looked like this : I showed my masterpiece to the grownups and I asked them if my drawing frightened them .
They answered me : “ Why would anyone be frightened by a hat ? ”
My drawing was not of a hat .
It showed a boa constrictor digesting an elephant .
I then drew the inside of the boa constrictor , so that the grownups could understand .
They always need to have things explained .
My Drawing Number two looked like this : The grownups 

And perhaps with a hint of sadness , he added : “ Straight ahead you can ' t go far ... ”


In [19]:
# activation generation shuffle
output = []
for index_batch, batch in enumerate(batches_shuffle):
    batch = batch.strip() # Remove trailing character

    batch = '<|endoftext|> ' + batch + ' <|endoftext|>'
    tokenized_text = tokenizer.tokenize(batch, add_prefix_space=False)
    inputs_ids = torch.tensor([tokenizer.convert_tokens_to_ids(tokenized_text)])

    #print('input shape: ', inputs_ids.shape)
    #print(batch)
    #print(tokenized_text)
    
    mapping = utils.match_tokenized_to_untokenized(tokenized_text, batch)

    #print('Mapping:')
    #for key in mapping.keys():
    #    print(batch.split()[key], ''.join([tokenized_text[i] for i in mapping[key]]))
    #print('A priori Token of interest:', tokenized_text[indexes_shuffle[index_batch][0]:indexes_shuffle[index_batch][1]])
         
    with torch.no_grad():
        encoded_layers = extractor_shuffle.model(inputs_ids) # last_hidden_state, pooler_output, hidden_states, attentions

        hidden_states_activations_ = np.vstack(encoded_layers[2]) # retrieve all the hidden states (dimension = layer_count * len(tokenized_text) * feature_count)

        #print('nb of layer:', len(encoded_layers[2]))
        #print('output shape:', hidden_states_activations_.shape)
        
        new_activations = []
        key_start = None
        key_stop = None
        
        #print('Mapping:')
        #for key in mapping.keys():
        #    print(batch.split()[key], ''.join([tokenized_text[i] for i in mapping[key]]))
        #print('A priori Token of interest:', tokenized_text[indexes_masked_tmp[index_batch][0]:indexes_masked_tmp[index_batch][1]])
            
        for key_, value in mapping.items(): 
            if (value[0] - 1) == (indexes_shuffle[index_batch][0]): #because we added [CLS] token at the beginning
                key_start = key_
        for key_, value in mapping.items(): 
            if value[-1] == (indexes_shuffle[index_batch][1]): #because we added [CLS] token at the beginning
                key_stop = key_

        #print(key)
        #print('dimension match:', len(tokenized_text)==hidden_states_activations_.shape[1])
        tmp = ' '.join([tokenizer.decode(tokenizer.convert_tokens_to_ids([tokenized_text[word] for word in mapping[index]])) for index in range(key_start, key_stop + 1)])
        tmp = tmp.replace('  ', ' ').strip()
        #print('Extracting sentence:')
        print(tmp)
        output.append(tmp)
        
        for word_index in range(key_start, key_stop + 1):
            word_activation = []
            word_activation.append([hidden_states_activations_[:,index, :] for index in mapping[word_index]])
            word_activation = np.vstack(word_activation)
            new_activations.append(np.mean(word_activation, axis=0).reshape(1,-1))

        
        #print(np.vstack(new_activations).shape)
        #if input()!='':
        #    break


assert ' '.join(output) == ' '.join(iterator_list[0])


Once
,
when
I
was
six
years
old
,
I
saw
a
magnificent
picture
in
a
book
about
the
primeval
forest
called
‘
Real
-
life
Stories
.
’
It
showed
a
boa
constrictor
swallowing
a
wild
animal
.
Here
is
a
copy
of
the
drawing
.
It
said
in
the
book
:
“
Boa
constrictors
swallow
their
prey
whole
,
without
chewing
.
Then
they
are
not
able
to
move
,
and
they
sleep
for
the
six
months
it
takes
for
digestion
.
”
So
I
thought
a
lot
about
the
adventures
of
the
jungle
and
,
in
turn
,
I
managed
,
with
a
coloured
pencil
,
to
make
my
first
drawing
.
My
Drawing
Number
one
.
It
looked
like
this
:
I
showed
my
masterpiece
to
the
grownups
and
I
asked
them
if
my
drawing
frightened
them
.
They
answered
me
:
“
Why
would
anyone
be
frightened
by
a
hat
?
”
My
drawing
was
not
of
a
hat
.
It
showed
a
boa
constrictor
digesting
an
elephant
.
I
then
drew
the
inside
of
the
boa
constrictor
,
so
that
the
grownups
could
understand
.
They
always
need
to
have
things
explained
.
My
Drawing
Number
two
looked
like
this
:
The
grownups


said
gravely
:
“
That
doesn
’
t
matter
;
where
I
live
,
everything
is
so
small
!
”
And
perhaps
with
a
hint
of
sadness
,
he
added
:
“
Straight
ahead
you
can
'
t
go
far
...
”


In [45]:
def transform_sentence_and_context(
    iterator, 
    past_context_size, 
    pretrained_model,
    transformation='shuffle',
    vocabulary=None,
    dictionary=None,
    select=None,
    seed=1111,
    add_prefix_space=True):
    """ Given a list of sentences, for each word, we transform its context outside a certain context window.
    Args:
        - iterator: list (of str)
        - context_size: int
        - pretrained_model: str
        - vocabulary: list
        - dictionary: dict
        - seed: int
        - add_prefix_space: bool
    Returns:
        - batch_tmp: list (of str)
        - index_tmp: list (of tuple of int)
    """
    random.seed(seed)
    punctuation = ['.', '!', '?', '...', '\'', ',', ';', ':', '/', '-', '"', '‘', '’', '(', ')', '{', '}', '[', ']', '`', '“', '”', '—']
    if select is None:
        words = ' '.join(iterator).split()
    else:
        words = iterator[select].split()
        
    all_words = ' '.join(iterator).split()
    words_before = [] if select is None else ' '.join(iterator[:select]).split()
    supp_before = [len([word for word in all_words[max(j+len(words_before)+1-past_context_size, 0):j+len(words_before)+1] if word in punctuation]) for j in range(len(words))] # we do not count punctuation in the number of words to shuffle

    # For each word, we compute the index of the other words to transform
    # We transform past context. Change conditions "i<j" and ... to something else if needed
    index_words_list_before = [[i for i, item in enumerate(all_words) if item not in punctuation if ((i!=(j+len(words_before))) and  (i <= j+len(words_before)-past_context_size-supp_before[j]))] for j in range(len(words))] # '<=' because context_size of 1 is the current word

    # Create the new array of sentences with original words 
    new_words = np.tile(np.array(all_words.copy()), (len(words), 1))

    for i in range(len(new_words)):
        if len(index_words_list_before[i])>0: # if there are words to change...
            if transformation=='shuffle':
                # Replace words that need to be shuffled by the random sampling (except fix point and punctuation)
                new_order = random.sample(index_words_list_before[i], len(index_words_list_before[i]))
                if len(index_words_list_before[i])>1:
                    while new_order==index_words_list_before[i]:
                        new_order = random.sample(index_words_list_before[i], len(index_words_list_before[i]))
                new_words[i, index_words_list_before[i]] = new_words[i, new_order]
            elif transformation=='pos_replacement':
                # Replace words that need to be replaced by words with same POS (except fix point and punctuation)
                new_words[i, index_words_list_before[i]] = pick_pos_word(new_words[i, index_words_list_before[i]], dictionary)
            elif transformation=='random_replacement':
                # Replace words that need to be replaced by random words (except fix point and punctuation)
                new_words[i, index_words_list_before[i]] = pick_random_word(new_words[i, index_words_list_before[i]], vocabulary)

    # Convert array to list
    new_words = list(new_words)
    new_words = [list(item) for item in new_words]
    batch_tmp = []
    index_tmp = []
    tokenizer = GPT2Tokenizer.from_pretrained(pretrained_model) # to replace with tokenizer of interest
    # adding transformed context to each sentence
    for i, sentence in enumerate(new_words):
        batch_tmp.append(' '.join(sentence).strip())
        # Determining associated indexes
        tmp1 = ' '.join(sentence[:i+len(words_before)])
        tmp2 = ' '.join(sentence[:i+len(words_before)+1])
        index_tmp.append((len(tokenizer.tokenize(tmp1.strip(), add_prefix_space=add_prefix_space)), 
                     len(tokenizer.tokenize(tmp2.strip(), add_prefix_space=add_prefix_space))
                    )) # to replace with tokenizer of interest and arguments
    return batch_tmp, index_tmp

In [48]:
iterator= iterator_list[0]
number_of_sentence=config['number_of_sentence']
number_sentence_before=config['number_of_sentence_before']
pretrained_model='gpt2'
past_context_size=config['attention_length_before']
transformation='shuffle'
vocabulary=None
dictionary=None
seed=1111
max_length=512
add_prefix_space=True


iterator = [item.strip() for item in iterator]
max_length -= 3 # for special tokens, because there is a G (dot) before last special token
assert number_of_sentence > 0
tokenizer = GPT2Tokenizer.from_pretrained(pretrained_model)

batch = []
indexes = []
sentence_count = 0
n = len(iterator)


print('entering while loop...')
# rest of the iterator + context 
while sentence_count < n:
    start = max(sentence_count - number_sentence_before, 0)
    stop = min(sentence_count + number_of_sentence, n)
    token_count = len(tokenizer.tokenize(' '.join(iterator[start:stop]), add_prefix_space=add_prefix_space)) # to replace with tokenizer of interest and arguments
    if token_count > max_length:
        raise ValueError('Cannot fit context with additional sentence. You should reduce context length.')
    # computing batch and indexes
    print(len(iterator[start:stop]))
    print(iterator[start:stop])
    
    batch_tmp, index_tmp = transform_sentence_and_context(
        iterator[start:stop], 
        past_context_size=past_context_size,
        pretrained_model=pretrained_model,
        transformation=transformation,
        vocabulary=vocabulary,
        dictionary=dictionary,
        select=stop-start-1,
        seed=seed,
        add_prefix_space=add_prefix_space
    )
    print('result:')
    for b in batch_tmp:
        print(b)
    batch += batch_tmp
    indexes += index_tmp
    sentence_count = stop
    print(stop-start-1)
    if input()!='':
        break

#for b in batch:
#    print(b)
#print(indexes)

entering while loop...
1
['Once , when I was six years old , I saw a magnificent picture in a book about the primeval forest called ‘ Real - life Stories . ’']
result:
Once , when I was six years old , I saw a magnificent picture in a book about the primeval forest called ‘ Real - life Stories . ’
Once , when I was six years old , I saw a magnificent picture in a book about the primeval forest called ‘ Real - life Stories . ’
Once , when I was six years old , I saw a magnificent picture in a book about the primeval forest called ‘ Real - life Stories . ’
Once , when I was six years old , I saw a magnificent picture in a book about the primeval forest called ‘ Real - life Stories . ’
when , Once I was six years old , I saw a magnificent picture in a book about the primeval forest called ‘ Real - life Stories . ’
I , Once when was six years old , I saw a magnificent picture in a book about the primeval forest called ‘ Real - life Stories . ’
I , was Once when six years old , I saw a magn

result:
a , in Stories a It swallowing years , is the animal I called magnificent life showed was book six forest Here ‘ saw - when picture . ’ I Real boa the copy constrictor primeval wild a . a Once of old about a drawing . It said in the book : “ Boa constrictors swallow their prey whole , without chewing .
swallowing , forest book saw drawing about I , when wild years boa called Stories old in magnificent a picture constrictor Real ‘ the - primeval I . ’ a It of six Here the a a a . showed was animal life Once copy is . It said in the book : “ Boa constrictors swallow their prey whole , without chewing .
saw , about showed called a drawing magnificent , copy years of the picture wild a Real Here boa I a is ‘ animal - swallowing It . ’ a was Once Stories It six forest life primeval . in a old when constrictor the I . book said in the book : “ Boa constrictors swallow their prey whole , without chewing .
about , It forest Real constrictor magnificent called , said is I six picture a 